In [127]:
from keras.preprocessing.image import ImageDataGenerator
from os import listdir
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.constraints import maxnorm
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [128]:
train_data_folder=['character_1_ka','character_2_kha','character_3_ga','character_4_gha','character_5_kna']

In [129]:
test_data_folder=['character_1_ka','character_2_kha','character_3_ga','character_4_gha','character_5_kna']

In [130]:
Label = ['Ka','Kha','Ga','Gha','Kna']
Labels = [0,1,2,3,4]

In [132]:

images=[]
y_train=[]
i=0

    
def load_photos(train_data_folder,Desktop):
    i=0
    y=[]
    images=[]
    for directory in train_data_folder:
        y_val=Labels[i]
        
        for name in listdir(Desktop+'/'+ directory):
            # load an image from file
            filename = Desktop+'/'+directory + '/' + name
            image = load_img(filename) #target_size=(32, 32,1))
            # convert the image pixels to a numpy array
            image = img_to_array(image)
            # reshape data for the model
            image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
            # prepare the image for the VGG model
            image = preprocess_input(image)

            images.append(image)
            y.append(y_val)
        i=i+1;
    return images,y

# load images
Train_Desktop = 'Train_val'
images,y = load_photos(train_data_folder,Train_Desktop)
print('Loaded Images: %d' % len(images))

Loaded Images: 8500


In [73]:
def encode_data(y):
    from sklearn.preprocessing import OneHotEncoder
    onehot_encoder = OneHotEncoder(sparse=False)
    y_train=y_train.reshape(8500,1)
    onehot_encoded = onehot_encoder.fit_transform(y_train)

In [133]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2)


In [140]:
import numpy as np
x_train = np.asarray(images)
y_train = np.asarray(y)

In [141]:
x_train = x_train / 255.0


In [143]:
datagen.fit(x_train)

In [51]:
input_shape = (32, 32, 3)
# Making sure that the values are float so that we can get decimal points after division
#x_train = x.astype('float32')
#y_train = y.astype('float32')

In [178]:
###CREATING THE MODEL
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape, padding='same', activation='relu'))#, kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
#model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
lrate = 0.001
decay = lrate/10


In [ ]:
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='sparse_categorical_crossentropy',  optimizer=sgd, metrics=['accuracy'])

In [160]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_27 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 8192)              0         
_________________________________________________________________
dense_51 (Dense)             (None, 512)               4194816   
_________________________________________________________________
dense_52 (Dense)             (None, 5)                 2565      
Total para

In [161]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=10,shuffle = True)

Epoch 1/10
266/265 [==============================] - 96s 361ms/step - loss: 1.4008 - acc: 0.4070
Epoch 2/10
266/265 [==============================] - 93s 348ms/step - loss: 1.0731 - acc: 0.5883
Epoch 3/10
266/265 [==============================] - 90s 340ms/step - loss: 0.7507 - acc: 0.7281
Epoch 4/10
266/265 [==============================] - 93s 351ms/step - loss: 0.5782 - acc: 0.7942
Epoch 5/10
266/265 [==============================] - 98s 367ms/step - loss: 0.4720 - acc: 0.8350
Epoch 6/10
266/265 [==============================] - 96s 361ms/step - loss: 0.4224 - acc: 0.8514
Epoch 7/10
266/265 [==============================] - 96s 361ms/step - loss: 0.3710 - acc: 0.8755
Epoch 8/10
266/265 [==============================] - 94s 355ms/step - loss: 0.3236 - acc: 0.8953
Epoch 9/10
266/265 [==============================] - 92s 346ms/step - loss: 0.2962 - acc: 0.9003
Epoch 10/10
266/265 [==============================] - 91s 340ms/step - loss: 0.2853 - acc: 0.9032


In [162]:
Desktop_test = 'Test'
images_test,y_test = load_photos(test_data_folder,Desktop_test)
print('Loaded Images Test: %d' % len(images_test))

Loaded Images Test: 1500


In [163]:
import numpy as np
x_test = np.asarray(images_test)
y_test = np.asarray(y_test)

In [164]:
score_SGD = model.evaluate_generator(datagen.flow(x_test, y_test, batch_size=32),steps=len(x_test)/32 )
print ("Accuracy with SGD : " , score_SGD[1])

Accuracy with SGD :  0.6113333334128062


In [176]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=15,shuffle = True)

Epoch 1/15
266/265 [==============================] - 106s 400ms/step - loss: 1.2363 - acc: 0.5047
Epoch 2/15
266/265 [==============================] - 103s 388ms/step - loss: 0.4521 - acc: 0.8431
Epoch 3/15
266/265 [==============================] - 107s 402ms/step - loss: 0.2887 - acc: 0.9024
Epoch 4/15
266/265 [==============================] - 112s 421ms/step - loss: 0.2162 - acc: 0.9292
Epoch 5/15
266/265 [==============================] - 110s 414ms/step - loss: 0.1758 - acc: 0.9422
Epoch 6/15
266/265 [==============================] - 113s 426ms/step - loss: 0.1466 - acc: 0.9515
Epoch 7/15
266/265 [==============================] - 103s 388ms/step - loss: 0.1473 - acc: 0.9500
Epoch 8/15
266/265 [==============================] - 112s 419ms/step - loss: 0.1145 - acc: 0.9626
Epoch 9/15
266/265 [==============================] - 85s 318ms/step - loss: 0.1190 - acc: 0.9609
Epoch 10/15
266/265 [==============================] - 81s 305ms/step - loss: 0.1068 - acc: 0.9657
Epoch 11/15

In [177]:
score_ADAM = model.evaluate_generator(datagen.flow(x_test, y_test, batch_size=32),steps=len(x_test)/32)
print ("Accuracy with ADAM : " , score_ADAM[1])

Accuracy with ADAM :  0.9093333328564962


In [179]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

model.fit_generator(datagen.flow(x_train, y_train, batch_size=32),
                    steps_per_epoch=len(x_train) / 32, epochs=32,shuffle = True)

Epoch 1/32
266/265 [==============================] - 130s 490ms/step - loss: 1.1379 - acc: 0.5502
Epoch 2/32
266/265 [==============================] - 122s 457ms/step - loss: 0.4501 - acc: 0.8416
Epoch 3/32
266/265 [==============================] - 123s 464ms/step - loss: 0.3079 - acc: 0.8942
Epoch 4/32
266/265 [==============================] - 122s 459ms/step - loss: 0.2333 - acc: 0.9217
Epoch 5/32
266/265 [==============================] - 123s 463ms/step - loss: 0.1923 - acc: 0.9359
Epoch 6/32
266/265 [==============================] - 122s 457ms/step - loss: 0.1733 - acc: 0.9464
Epoch 7/32
266/265 [==============================] - 121s 456ms/step - loss: 0.1562 - acc: 0.9490
Epoch 8/32
266/265 [==============================] - 125s 469ms/step - loss: 0.1431 - acc: 0.9502
Epoch 9/32
266/265 [==============================] - 122s 458ms/step - loss: 0.1256 - acc: 0.9582
Epoch 10/32
266/265 [==============================] - 123s 461ms/step - loss: 0.1335 - acc: 0.9572
Epoch 11/

In [180]:
score_ADAM = model.evaluate_generator(datagen.flow(x_test, y_test, batch_size=32),steps=len(x_test)/32)
print ("Accuracy with ADAM : " , score_ADAM[1])

Accuracy with ADAM :  0.9566666669845582
